In [ ]:
# enjoy ! :D

import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm

from astropy.visualization import hist
from astropy.stats import histogram

from astropy.io import fits

import glob #for the fits
from pathlib import Path
import os

from astropy import units as u
from ccdproc import Combiner, CCDData

In [ ]:
# creates a new txt file with updated names for each call. also adds the IRAF required 'imagetyp' header section with correct name.
# double check the detected headers (under # new suffix) match what your files will contain, or however the observer formatted them.

# make a list of files

def list_files(folder, extension):
    folder = Path(folder)
    if not folder.exists():
        print(f"Directory {folder} not found")
        return[]

    # gets rid of whitespace for iraf.

    print(f"Working in: {folder}")
    i = 0

    # reading names
    for item in folder.rglob(f"*{extension}"):

        if item.is_file():

            new_name = item.name.replace(" ", "")

            #renaming
            if new_name != item.name:
                new_path = item.with_name(new_name)
                os.rename(item, new_path)
                #print(f"Renamed {item} to {new_path}")
                i += 1

    print(f"Removed whitespace from {i} filenames in {folder}")
    return [file.name for file in folder.iterdir() if file.is_file() and file.suffix == extension]

# renames all the files

def rename_fits_files(file_list, folder, output_log = "current_files.txt"):

    # setting paths
    folder = Path(folder)
    log_path = folder / output_log

    # checking / making directories
    if not folder.exists():
        print(f"Directory {folder} does not exist. Creating now.")
        folder.mkdir(parents=True, exist_ok=True)

    print(f"Working in: {folder} \n")

    # going through the names

    for file_name in file_list:

        file_path = folder / file_name

        try:
            with fits.open(file_path, mode = "update") as hdul:
                header = hdul[0].header
                file_type = header.get("IMAGETYP", "").lower()

                # picking a file naming suffix
                if "bias" in file_type or "zero" in file_type:
                    suffix = ".bias"
                    header["IMAGETYP"] = "zero"
                elif "dark" in file_type:
                    suffix = ".dark"
                    header["IMAGETYP"] = "dark"
                elif "flat" in file_type:
                    suffix = ".flat"
                    header["IMAGETYP"] = "flat"
                elif "object" in file_type:
                    suffix = ".target"
                elif "light" in file_type:
                    suffix = ".target"
                else:
                    suffix = ".unknown"

                hdul.flush()

            # making new file name
            new_name = f"{file_path.stem}{suffix}.fits"
            new_path = folder / new_name

            # renaming
            if any(x.lower() in file_name.lower() for x in ["bias", "flat", "dark", "target", "unknown"]):
                print("For file " + file_name + ": No changes, file type / extension already present. \n")
            else:
                os.rename(file_path, new_path)
                print(f"Renamed {file_name} -> {new_name}")

        except Exception as e:
            print(f"Error processing {file_name}: {e}")
            print(f"Might want to inspect. \n")

    global current_files
    current_files = list_files(folder, ".fits")

    with open(log_path, "w") as log_file:
        for file in current_files:
            log_file.write(file + "\n")

    print(f"Updated list of file names in: {output_log}")

In [ ]:
# if you are working from the Linux shared data drive, this will help you define the right file path.
# note that from a Jupyter notebook, it resolves to the directory from which you launched the notebook and then the file folder.

relative = Path("/data/cartermg/329/final/V927Her")
current_path = os.getcwd()

print(relative)
print(current_path)

/data/cartermg/329/final/V927Her
/data/cartermg/329


In [ ]:
# enter file location path, if above is incorrect.

in_folder = relative

# check the correct file extensions.

current_files = list_files(in_folder, ".fit")

# the global variable current_files only updates in the rename_fits_files()function.
# print(current_files)

Working in: /data/cartermg/329/final/V927Her
Removed whitespace from 240 filenames in /data/cartermg/329/final/V927Her


In [ ]:
# main executable.

rename_fits_files(current_files, in_folder)

Working in: /data/cartermg/329/final/V927Her 

For file Bias.30878.fit: No changes, file type / extension already present. 

For file Bias.30879.fit: No changes, file type / extension already present. 

For file Bias.30880.fit: No changes, file type / extension already present. 

For file Bias.30881.fit: No changes, file type / extension already present. 

For file Bias.30882.fit: No changes, file type / extension already present. 

For file Bias.30883.fit: No changes, file type / extension already present. 

For file Bias.30884.fit: No changes, file type / extension already present. 

For file Dark20s.30885.fit: No changes, file type / extension already present. 

For file Dark20s.30886.fit: No changes, file type / extension already present. 

For file Dark20s.30887.fit: No changes, file type / extension already present. 

For file Dark20s.30888.fit: No changes, file type / extension already present. 

For file Dark20s.30889.fit: No changes, file type / extension already present. 

Fo